- Variables compartidas en Spark:
    - Broadcast variables
    - Accumulators
- Partitioning

Broadcast variables, son variables solo de lectura grabadas por nuestro Spark Driver y puestas a disposición de todos los nodos de trabajo en el cluster.
Esta distribución de información las hace disponibles a cualquier tarea corriendo en los ejecutores de los nodos. Se comparten a través de un protocolo P2P (basado en BitTorrent / BitTorrent Broadcast), lo que proporciona gran escalabilidad simplemente al ser enviadas mediante el Spark Driver.

![broadcast vars](https://github.com/israelzuniga/dlatam-bigdata-workshop/blob/be9dbf6b5fcf4f0b32c2f490a4045a5f2c6daa3c/notebooks/img/broadcast_vars.png?raw=true)




**Performance and Scalability of Broadcast in Spark, Mosharaf Chowdhury. University of California, Berkeley:**

[Performance and Scalability of Broadcast in Spark](https://pdfs.semanticscholar.org/7b0e/6a3dc18babb19daddb63890e763795943485.pdf)

In [1]:
APP_NAME = "API_Avanzado"
SPARK_URL = "local[*]"


from pyspark import SparkConf, SparkContext

In [2]:
conf = (SparkConf()\
        .setMaster(SPARK_URL)\
        .setAppName(APP_NAME))



sc = SparkContext(conf= conf)

## Cómo crear y usar variables de broadcast? `broadcast()`


`SparkContext.broadcast(value)`

In [4]:
stations = sc.broadcast({'83': 'Mezes Park', '84':'Dolores Park'})

In [5]:
stations

**Aunque podemos instanciar variables de broadcast a partir de archivos; su posible uso se traslapa con la función básica del RDD** 👮



Una vez que tenemos objetos de tipo broadcast, los podemos usar con la siguiente sintaxis:

`Clase: value()`

`Broadcast.value()`


In [9]:
stations.value['84']

'Dolores Park'

-----
`unpersist()`

Igual que la función anterior de persistencia para RDDs.

`Broadcast.unpersist(blocking=False)`

In [10]:
stations.unpersist()

## Cuáles son las ventajas de las variables de Broadcast ?


👀


------


# Accumulators

Otro tipo de variables compartidas en Spark, son las variables de tipo acumulador. A diferencia del tipo Broadcast, estas pueden actualizar su valor. Especificamente son valores numéricos (enteros o decimales) que se pueden incrementar.

Se escriben inicialmente por nuestro Spark Driver y su actualización corresponde a las tareas en los nodos. El valor final del acumulador puede ser obtenido de regreso, generalmente al acabar la app de Spark.

Se actualizan solamente una vez por tarea exitosa. Los nodos envian las actualizaciones hacia el Spark Driver, que es el único proceso que puede leer su valor.

In [11]:
acc = sc.accumulator(0)

In [12]:
def plus_one(x):
    global acc
    acc += 1
    return x + 1

In [13]:
myrdd = sc.parallelize([1, 2, 3, 4, 5])

In [14]:
myrdd.map(lambda x: plus_one(x)).collect()

[2, 3, 4, 5, 6]

In [15]:
print('Registros procesados: ' + str(acc.value))

Registros procesados: 5


In [16]:
myrdd.map(lambda x: plus_one(x)).collect()

[2, 3, 4, 5, 6]

In [17]:
print('Registros procesados: ' + str(acc.value))

Registros procesados: 10


### Custom Accumulators `Obj: AccumulatorParam`

In [18]:
from pyspark import AccumulatorParam

In [19]:
class VectorAccumulatorParam(AccumulatorParam):
    def zero(self, value):
        dict1 = {}
        for i in range(0, len(value)):
            dict1[i] = 0
        return dict1
    # next funct
    def addInPlace(self, val1, val2):
        for i in val1.keys():
            val1[i] += val2[i]
        return val1

In [20]:
rdd1 = sc.parallelize([{ 0: 0.3, 1: 0.8, 2: 0.4}, {0: 0.2, 1: 0.4, 2: 0.2}])

In [21]:
vector_acc = sc.accumulator({0: 0, 1: 0, 2: 0}, VectorAccumulatorParam())

In [22]:
def mapping_fn(x):
    global vector_acc
    vector_acc += x
    

In [23]:
rdd1.foreach(mapping_fn)

In [24]:
vector_acc.value

{0: 0.5, 1: 1.2000000000000002, 2: 0.6000000000000001}

## Usos para variables tipo Accumulator

Generalmente los usaremos para propósitos operacionales, como:

- conteo de registros procesados
- seguimiento de registros malformados